In [1]:
library(tidyverse)
library(kableExtra)
rsv <- read_csv("rsv.csv")
str(rsv) %>% print

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.2 ──
✔ ggplot2 3.4.2     ✔ purrr   1.0.1
✔ tibble  3.2.1     ✔ dplyr   1.1.2
✔ tidyr   1.3.0     ✔ stringr 1.5.0
✔ readr   2.1.4     ✔ forcats 1.0.0
Warning message:
“package ‘ggplot2’ was built under R version 4.2.3”
Warning message:
“package ‘tibble’ was built under R version 4.2.3”
Warning message:
“package ‘dplyr’ was built under R version 4.2.3”
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Attaching package: ‘kableExtra’


The following object is masked from ‘package:dplyr’:

    group_rows


Rows: 1374 Columns: 12
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
dbl (12): id, age, sex, bwt, smoke, atopy, fhxasth, resp, wheeze, cyan, los,...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `

spc_tbl_ [1,374 × 12] (S3: spec_tbl_df/tbl_df/tbl/data.frame)
 $ id     : num [1:1374] 1 2 3 4 5 6 7 8 9 10 ...
 $ age    : num [1:1374] 1 2 12 9 4 3 2 12 3 12 ...
 $ sex    : num [1:1374] 0 1 0 1 0 1 0 0 1 1 ...
 $ bwt    : num [1:1374] 3 3 2 3.1 2.8 1.6 2.8 3.4 2 3.5 ...
 $ smoke  : num [1:1374] 0 1 0 1 1 1 1 1 1 0 ...
 $ atopy  : num [1:1374] 0 0 0 0 0 0 0 0 0 0 ...
 $ fhxasth: num [1:1374] 1 0 0 0 0 0 1 0 0 1 ...
 $ resp   : num [1:1374] 60 60 38 58 44 30 47 35 51 35 ...
 $ wheeze : num [1:1374] 0 1 0 2 1 1 2 1 2 1 ...
 $ cyan   : num [1:1374] 2 0 0 0 0 0 1 0 0 0 ...
 $ los    : num [1:1374] 6 5 10 5 5 8 16 4 2 4 ...
 $ mmi    : num [1:1374] 1 1 1 1 1 1 1 1 0 1 ...
 - attr(*, "spec")=
  .. cols(
  ..   id = col_double(),
  ..   age = col_double(),
  ..   sex = col_double(),
  ..   bwt = col_double(),
  ..   smoke = col_double(),
  ..   atopy = col_double(),
  ..   fhxasth = col_double(),
  ..   resp = col_double(),
  ..   wheeze = col_double(),
  ..   cyan = col_double(),
  ..   lo

(a) Investigate the association between age and length of hospital stay.

In [2]:
##(a)
cor(rsv$age,rsv$los) %>% print

[1] -0.223165


(b) Establish a range of typical values for length of hospital stay for children with the following covariate profile:
• Eight months of age. • Male.
• No wheezing.

In [3]:
rsv %>%
    filter(age == 8,sex == 1) %>%
    summarise(lower = quantile(los,0.25),
                upper = quantile(los,0.75))


lower,upper
<dbl>,<dbl>
3,5


In [4]:
rsv %>%
    filter(wheeze == 0) %>%
    summarise(lower = quantile(los,0.25),
                upper = quantile(los,0.75))

lower,upper
<dbl>,<dbl>
3,7


(c) Develop a prediction model for major medical intervention based on the following variables: 

• Age.
• Sex.
• Birth weight.
• Prior smoke exposure.
• Atopy.
• Family history of asthma. • Respiratory rate.
• Wheezing.
• Cyanosis.

In [50]:
## when he said develop a model, does it mean I have free choice of the model?
library(caret)
library(glmnet)
set.seed(123123)
train_index = createDataPartition(rsv$los,p = 0.5,list = FALSE)
train_rsv = rsv[train_index,]
test_rsv = rsv[-train_index,]

model_1 = train(x = train_rsv %>% select(-los) %>% as.matrix,
                y = train_rsv$los,
                method = "lm",
                trControl = trainControl(method = "cv",number = 10),
                metric = "RMSE")


cv.glmnet(
                x = 
    model.matrix(los~age*sex*bwt*smoke*atopy*fhxasth*resp*as.factor(wheeze)*as.factor(cyan),
                             data = train_rsv)[,-1],
                y = train_rsv$los,
                family = "gaussian",
                type.measure = "mse",
                nfolds = 10,
)
model_2 = glmnet(                
    x = 
    model.matrix(los~age*sex*bwt*smoke*atopy*fhxasth*resp*as.factor(wheeze)*as.factor(cyan),
                             data = train_rsv)[,-1],
                y = train_rsv$los,
                alpha = 1,
                lambda = 0.1244)


Call:  cv.glmnet(x = model.matrix(los ~ age * sex * bwt * smoke * atopy *      fhxasth * resp * as.factor(wheeze) * as.factor(cyan), data = train_rsv)[,      -1], y = train_rsv$los, type.measure = "mse", nfolds = 10,      family = "gaussian") 

Measure: Mean-Squared Error 

    Lambda Index Measure    SE Nonzero
min 0.1244    39   10.25 1.196      42
1se 0.7288     1   10.82 1.202       0

In [51]:
## Predicting error
mean((predict(model_1,newdata = test_rsv) - test_rsv$los)^2)
mean((predict(model_2,newx = 
model.matrix(los~age*sex*bwt*smoke*atopy*fhxasth*resp*as.factor(wheeze)*as.factor(cyan),
                             data = test_rsv)[,-1] 
) - test_rsv$los)^2)

[1] 7.206809

[1] 11.53948

8. 20 pts You are working with an investigative team that focuses on the use of magnetic resonance imaging (MRI) to understand variation in properties of the spinal cord. One goal in particular is to estimate specific quantiles of spinal cord volume (mm3) in the population. As is often the case, there are many approaches that could be used to accomplish this goal. Consider the following two estimation approaches for estimating the pth quantile of a distribution based on a total of n independently sampled subjects:

• An estimate based on the presumption that spinal cord volume follows a normal distribution with mean μ and variance σ2: Q̂(p) = μ̂+Φ1(p)×̂σ, where μ̂ = Xn denotes the sample mean, ̂σ denotes the sample standard deviation, and Φ1(p) denotes the pth quantile of the standard normal distribution.

• An estimate that does not make specific assumptions about the underlying distribution of spinal cord volume, based on the inverse of the empirical cumulative distribution function. This can be obtained in R through the command quantile(..., type = 1). Specifying the ‘type’ argument is important, as the default approach is different from the one described here.

In this problem, your task is to compare these two estimation approaches using Monte Carlo (simulation) based techniques. Your code for all problems must be turned in as clearly labeled supplementary .R file.

In [ ]:
## (a) simulation
library(foreach)
set.seed(123123)
